In [23]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
from typing import Dict
import os
from pydantic import BaseModel
from pathlib import Path
from PyPDF2 import PdfReader
from pptx import Presentation
from PIL import Image 
from openai import OpenAI



In [24]:
load_dotenv(override=True)

True

In [25]:
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
client = OpenAI() 

In [26]:
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
llama = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
from agents import Agent, function_tool, Runner
from pathlib import Path
import os
import json
import shutil
from datetime import datetime
import pytz

STORAGE_ROOT = Path(os.getenv("STORAGE_ROOT", "../../storage")).resolve()
METADATA_FILE = STORAGE_ROOT / os.getenv("METADATA_FILE", "file_metadata.json")

STORAGE_ROOT.mkdir(parents=True, exist_ok=True)
if not METADATA_FILE.exists():
    with open(METADATA_FILE, "w") as f:
        json.dump([], f)


def load_metadata():
    if METADATA_FILE.exists():
        with open(METADATA_FILE, "r") as f:
            try:
                data = json.load(f)
                return data if isinstance(data, list) else []
            except json.JSONDecodeError:
                return []
    return []


def save_metadata(data):
    with open(METADATA_FILE, "w") as f:
        json.dump(data, f, indent=4)


def get_current_time():
    tz = pytz.timezone("Asia/Kolkata")
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")


def update_last_accessed(files):
    """Update last accessed time for the given metadata entries."""
    metadata = load_metadata()
    updated = False
    for entry in metadata:
        for f in files:
            if entry["path"] == f["path"]:
                entry["last_accessed"] = get_current_time()
                updated = True
    if updated:
        save_metadata(metadata)


# -----------------------------
# Tools (Wrapped for the Agent)
# -----------------------------

@function_tool
def update_file_tool(
    file_name: str,
    new_category: str = None,
    new_tag: str = None,
):
    """
    Updates file metadata and optionally moves file between categories.

    Arguments:
    - file_name: existing stored file name
    - new_category: if provided, moves file to this category folder
    - new_tag: if provided, updates the file's tag

    Notes:
    - If only new_tag is given → updates tag only.
    - If only new_category is given → moves file but keeps tag.
    - If both given → moves file and updates tag.
    """

    metadata = load_metadata()
    entry = next((m for m in metadata if m["name"] == file_name), None)

    if not entry:
        return {"error": f"File '{file_name}' not found in metadata."}

    src = Path(entry["path"])
    old_category_dir = src.parent  

    if new_category:
        new_category_dir = STORAGE_ROOT / new_category
        new_category_dir.mkdir(parents=True, exist_ok=True)

        dst = new_category_dir / src.name
        if dst.exists():
            return {"error": f"File already exists in destination: {dst}"}

        shutil.move(str(src), dst)
        entry["path"] = str(dst)
        entry["category"] = new_category

        try:
            if old_category_dir.exists() and not any(old_category_dir.iterdir()):
                old_category_dir.rmdir()
        except Exception as e:
            print(f"Warning: Could not delete empty folder {old_category_dir}: {e}")

    # --- Handle tag update ---
    if new_tag is not None:
        entry["tag"] = new_tag


    entry["last_accessed"] = get_current_time()


    save_metadata(metadata)

    return {
        "success": True,
        "file": entry["name"],
        "new_category": entry["category"],
        "new_tag": entry["tag"],
        "path": entry["path"]
    }


@function_tool
def add_file_tool(
    file_path: str,
    category: str,
    tag: str = None,
    name: str = None,
    overwrite: bool = False,
    description: str = None
):
    """
    Uploads a file into STORAGE_ROOT/category/tag/ (if tag provided)
    and records metadata.

    Arguments:
    - file_path: local path of the uploaded file
    - category: folder name inside STORAGE_ROOT
    - tag: optional label -> will also create subfolder inside category
    - name: optional new name for the file
    - overwrite: whether to overwrite if a file with same name exists
    - description: short summary of the file contents
    """
    src = Path(file_path).resolve()
    if not src.exists():
        return {"error": f"File not found: {src}"}

    if tag:
        target_dir = STORAGE_ROOT / category / tag
    else:
        target_dir = STORAGE_ROOT / category

    target_dir.mkdir(parents=True, exist_ok=True)

    if name:
        name = Path(name).stem
        final_name = f"{name}{src.suffix}"
    else:
        final_name = src.name

    dst = target_dir / final_name

    if dst.exists() and not overwrite:
        base, ext = dst.stem, dst.suffix
        i = 1
        while True:
            new_name = f"{base}_{i}{ext}"
            new_dst = target_dir / new_name
            if not new_dst.exists():
                dst = new_dst
                final_name = new_name
                break
            i += 1

    shutil.copy2(src, dst)

    metadata = load_metadata()
    entry = {
        "name": final_name,
        "path": str(dst),
        "category": category,
        "tag": tag,
        "description": description if description else "No description provided",
        "last_accessed": get_current_time(),
    }
    metadata.append(entry)
    save_metadata(metadata)

    return {
        "success": True,
        "stored_file": final_name,
        "category": category,
        "tag": tag,
        "description": entry["description"]
    }





@function_tool
def get_files_tool(name: str=None, category: str=None, tag: str=None, query: str=None):
    """
    Fetches files based on filters OR semantic query.
    """
    metadata=load_metadata()
    results=[]
    if query:
        for entry in metadata:
            if query.lower() in entry.get("description","").lower():
                entry["last_accessed"]=datetime.now().isoformat()
                results.append(entry)
    else:
        for entry in metadata:
            match=True
            if name and entry["name"]!=name:
                match=False
            if category and entry["category"]!=category:
                match=False
            if tag and entry["tag"]!=tag:
                match=False
            if match:
                entry["last_accessed"]=datetime.now().isoformat()
                results.append(entry)
    save_metadata(metadata)
    return {"files":results}


@function_tool
def load_metadata_tool():
    """Returns the current metadata.json content as a list of file entries."""
    return load_metadata()

@function_tool
def delete_files_tool(name: str=None, category: str=None, tag: str=None):
    """
    Deletes files and updates metadata based on filters:
    - category only
    - tag only
    - name only
    - name + category + tag
    - tag + category

    After deletion, empty folders (including tag and category folders) are removed.
    """
    metadata = load_metadata()
    updated_metadata = []
    deleted = []

    for entry in metadata:
        match = True
        if name and entry["name"] != name:
            match = False
        if category and entry["category"] != category:
            match = False
        if tag and entry["tag"] != tag:
            match = False

        if match:
            file_path = Path(entry["path"])
            if file_path.exists():
                try:
                    file_path.unlink()
                except Exception as e:
                    return {"error": f"Could not delete {file_path}: {e}"}
            deleted.append(entry)
        else:
            updated_metadata.append(entry)

    save_metadata(updated_metadata)

    # --- Recursive cleanup function ---
    def cleanup_empty_folders(root: Path):
        """Recursively remove empty folders under root, ignoring hidden files."""
        for folder in sorted(root.glob("**/*"), key=lambda x: len(x.parts), reverse=True):
            if folder.is_dir():
                if not any(f for f in folder.iterdir() if not f.name.startswith('.')):
                    try:
                        folder.rmdir()
                    except Exception:
                        pass

    # --- Clean from STORAGE_ROOT ---
    cleanup_empty_folders(STORAGE_ROOT)

    return {
        "deleted": [d["name"] for d in deleted],
        "remaining_files": [f["name"] for f in updated_metadata]
    }



@function_tool
def read_contents(file_path: str, max_pages: int = 20, max_chars: int = 10000):
    """
    Reads the contents of a given file and returns a text preview.

    Supports: PDF, PowerPoint, Text, JSON, CSV, Markdown, Images (AI description).
    """

    import base64
    from pathlib import Path
    from PyPDF2 import PdfReader
    from pptx import Presentation
    import json
    import csv

    path = Path(file_path)
    if not path.exists():
        return {"error": f"File not found: {file_path}"}

    text_content = ""

    try:
        if path.suffix.lower() == ".pdf":
            try:
                reader = PdfReader(str(path))
                for i, page in enumerate(reader.pages[:max_pages]):
                    text_content += f"\n--- Page {i+1} ---\n"
                    extracted = page.extract_text()
                    text_content += extracted if extracted else "[No extractable text]\n"
            except Exception as e:
                return {"error": f"PDF reading failed: {e}"}

        elif path.suffix.lower() in [".pptx", ".ppt"]:
            try:
                prs = Presentation(str(path))
                for i, slide in enumerate(prs.slides[:max_pages]):
                    text_content += f"\n--- Slide {i+1} ---\n"
                    for shape in slide.shapes:
                        if hasattr(shape, "text") and shape.text.strip():
                            text_content += shape.text + "\n"
            except Exception as e:
                return {"error": f"PowerPoint reading failed: {e}"}

        elif path.suffix.lower() in [".txt", ".md"]:
            try:
                text_content = path.read_text(errors="ignore")[:max_chars]
            except Exception as e:
                return {"error": f"Text file reading failed: {e}"}

        elif path.suffix.lower() == ".json":
            try:
                data = json.loads(path.read_text(errors="ignore"))
                text_content = json.dumps(data, indent=2)[:max_chars]
            except Exception as e:
                return {"error": f"JSON reading failed: {e}"}

        elif path.suffix.lower() == ".csv":
            try:
                with open(path, newline="", encoding="utf-8", errors="ignore") as csvfile:
                    reader = csv.reader(csvfile)
                    rows = []
                    for i, row in enumerate(reader):
                        if i >= max_pages:  
                            break
                        rows.append(", ".join(row))
                    text_content = "\n".join(rows)[:max_chars]
            except Exception as e:
                return {"error": f"CSV reading failed: {e}"}

       
        elif path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            try:
                with open(path, "rb") as img_file:
                    img_bytes = img_file.read()
                    img_b64 = base64.b64encode(img_bytes).decode("utf-8")

                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are an assistant that describes images in plain language."},
                        {"role": "user", "content": f"Describe this image:\n[base64 image data]\n{img_b64}"}
                    ],
                    max_tokens=300
                )

                text_content = response.choices[0].message.content.strip()

            except Exception as e:
                return {"error": f"Image description failed: {e}"}

        if len(text_content) > max_chars:
            text_content = text_content[:max_chars] + "... [truncated]"

        return {
            "file_name": path.name,
            "path": str(path),
            "content_preview": text_content if text_content.strip() else "[No readable content]"
        }

    except Exception as e:
        return {"error": f"Unexpected error: {e}"}

# -----------------------------
# The Agent
# -----------------------------
file_reader_agent = Agent(
    name="FileReaderAgent",
    instructions=(
        "You are a file classifier. "
        "1. Always read the file content using `read_contents(file_path)` and summarize it in 1–2 sentences. "
        "2. Use `load_metadata_tool()` to inspect existing files' descriptions, categories, and tags. "
        "3. Recommend the most appropriate **category** and **tag** for this file: "
        "   - The **category** should be a broad, high-level classification (e.g., 'reference_materials', 'novels', 'reports') in snake_case, lowercase, no spaces. "
        "   - The **tag** should be a more specific subtopic within that category (e.g., 'compiler_design', 'fiction', 'annual_report') in snake_case, lowercase, no spaces. "
        "   - Reuse an existing category/tag if it already fits the content; create a new one only if necessary. "
        "4. Only output the recommended category, tag, and 1–2 sentence description. "
        "   Do not modify metadata.json directly."
    ),
    tools=[read_contents, load_metadata_tool],
    model="gpt-4o-mini"
)


reader_tool = file_reader_agent.as_tool(
    tool_name="file_reader",
    tool_description="Reads a file, generates a short description, and suggests the most appropriate category and tag for it."
)

file_agent = Agent(
    name="FileManagerAgent",
    instructions=(
        "You are a file manager agent. "
        "Your main responsibility is to manage files: add, update, fetch metadata, and delete files. "
        "\n\n"
        "For file additions: "
        "- Always keep the original 'name' field exactly as the uploaded filename. "
        "- By default, call the `file_reader` tool to get the recommended description, category, and tag. "
        "- If the user explicitly provides a category and/or tag, use those values instead of calling `file_reader` for them. "
        "- If the user provides a description, use it; otherwise, use the one from `file_reader`. "
        "- Ensure that category and tag values are stored in snake_case (e.g., 'reference_materials', 'compiler_design'). "
        "- Never invent categories, tags, or descriptions yourself. "
        "\n\n"
        "For updating or moving files: "
        "- Only modify category, tag, or description if explicitly requested by the user. "
        "- Do not change the filename unless explicitly instructed. "
        "- Do not reclassify the file with `file_reader` unless the user requests reclassification."
        "\n\n"
        "For fetching files: "
        "- Analyze the metadata's description field to determine relevance. "
        "- Return files with all metadata fields (name, category, tag, path, description). "
        "- Never invent or assume content."
        "\n\n"
        "Always ground all decisions in metadata and outputs from `file_reader`. "
        "Never attempt to classify or describe files on your own."
    ),
    tools=[add_file_tool, get_files_tool, delete_files_tool, update_file_tool, reader_tool],
    model="gpt-4o-mini"
)


In [28]:
import time
import threading
import requests
from pathlib import Path
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

DOWNLOADS_DIR = Path.home() / "Downloads"
API_URL = "http://127.0.0.1:8000/chat"
processed_files = set()  # Track already processed files

def wait_for_file_ready(file_path: Path, timeout=20, poll_interval=0.5):
    start_time = time.time()
    last_size = -1
    while time.time() - start_time < timeout:
        if not file_path.exists():
            time.sleep(poll_interval)
            continue
        current_size = file_path.stat().st_size
        if current_size == last_size and current_size > 0:
            return True
        last_size = current_size
        time.sleep(poll_interval)
    return False

class DownloadHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return
        file_path = Path(event.src_path)

        # Ignore temp files
        if file_path.suffix in [".crdownload", ".part", ".tmp"]:
            return

        # Avoid processing the same file twice
        if file_path in processed_files:
            return
        processed_files.add(file_path)

        print(f"[NEW FILE DETECTED] {file_path}")

        if not wait_for_file_ready(file_path):
            print(f"⚠️ File not ready: {file_path}")
            return

        try:
            with open(file_path, "rb") as f:
                files = {"uploaded_files": (file_path.name, f)}
                response = requests.post(API_URL, files=files)

            if response.status_code == 200:
                resp_json = response.json()
                print(f"[AGENT RESPONSE] {resp_json.get('chat_reply', '')}")
                try:
                    file_path.unlink()
                    print(f"✅ Deleted original from Downloads: {file_path}")
                except FileNotFoundError:
                    print(f"⚠️ File already deleted: {file_path}")
            else:
                print(f"⚠️ API call failed: {response.status_code} {response.text}")

        except Exception as e:
            print(f"[ERROR running agent] {e}")

def start_download_monitor():
    if not DOWNLOADS_DIR.exists():
        print(f"Downloads folder not found: {DOWNLOADS_DIR}")
        return None

    event_handler = DownloadHandler()
    observer = Observer()
    observer.schedule(event_handler, str(DOWNLOADS_DIR), recursive=False)
    observer.start()
    print(f"🚀 Monitoring {DOWNLOADS_DIR} for new downloads...")

    def _keep_alive():
        try:
            while True:
                time.sleep(2)
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    thread = threading.Thread(target=_keep_alive, daemon=True)
    thread.start()
    return observer

# Start monitoring
observer = start_download_monitor()


Unhandled exception in FSEventsEmitter
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/watchdog/observers/fsevents.py", line 307, in run
    _fsevents.add_watch(self, self.watch, self.events_callback, self.pathnames)
RuntimeError: Cannot add watch <ObservedWatch: path='/Users/vishwajithp/Downloads', is_recursive=False> - it is already scheduled


🚀 Monitoring /Users/vishwajithp/Downloads for new downloads...


In [29]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse, FileResponse, HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path
import shutil
import asyncio
import json
import nest_asyncio
import uvicorn
from agents.items import ToolCallOutputItem
from mimetypes import guess_type
from typing import Optional

nest_asyncio.apply()
app = FastAPI(title="File Manager Agent")

# -----------------------------
# STORAGE SETUP
# -----------------------------
STORAGE_ROOT = Path("../../storage").resolve()
STORAGE_ROOT.mkdir(parents=True, exist_ok=True)
UPLOADS_DIR = STORAGE_ROOT / "uploads"
UPLOADS_DIR.mkdir(exist_ok=True)

METADATA_FILE = STORAGE_ROOT / "file_metadata.json"
if not METADATA_FILE.exists():
    METADATA_FILE.write_text("[]")  # initialize empty metadata


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],       
    allow_credentials=True,
    allow_methods=["*"],       
    allow_headers=["*"], 
)

# -----------------------------
# Helper functions
# -----------------------------
def load_metadata():
    try:
        with open(METADATA_FILE, "r") as f:
            data = json.load(f)
            return data if isinstance(data, list) else []
    except json.JSONDecodeError:
        return []
def fetch_files_for_api(name: str=None, category: str=None, tag: str=None):
    """
    Fetches a file asked by the user and sends it back to him.
    Lightweight version for API use only.
    """
    metadata=load_metadata()
    results=[]
    for entry in metadata:
        match=True
        if name and entry["name"]!=name:
            match=False
        if category and entry["category"]!=category:
            match=False
        if tag and entry["tag"]!=tag:
            match=False
        if match:
            results.append(entry)
    return {
        "files":[
            {"name":f["name"],"category":f["category"],"tag":f.get("tag"),"path":f["path"]}
            for f in results
        ]
    }
# -----------------------------
# Agent runner (async-safe)
# -----------------------------
def run_agent_sync(user_input: str):
    """Run the agent in a sync context safely"""
    from agents import Runner

    try:
        loop = asyncio.get_running_loop()
        result = loop.run_until_complete(Runner.run(file_agent, user_input))
    except RuntimeError:
        result = asyncio.run(Runner.run(file_agent, user_input))

    files_to_show = []
    for item in result.new_items:
        if isinstance(item, ToolCallOutputItem) and isinstance(item.output, dict):
            if "files" in item.output:
                files_to_show.extend([Path(f["path"]) for f in item.output["files"] if "path" in f])

    return result.final_output, files_to_show

# -----------------------------
# Chat endpoint
# -----------------------------
@app.post("/chat")
async def chat_endpoint(
    message: Optional[str] = Form(None),
    uploaded_files: list[UploadFile] = File(None)
):
    file_paths = []

    # Save uploaded files
    if uploaded_files:
        for uploaded_file in uploaded_files:
            file_path = UPLOADS_DIR / Path(uploaded_file.filename).name
            with open(file_path, "wb") as f:
                shutil.copyfileobj(uploaded_file.file, f)
            file_paths.append(file_path)

    # Load metadata and include in prompt
    metadata = load_metadata()
    metadata_json = json.dumps(metadata, indent=2)

    user_input = f"Here is the current file metadata (name, category, tag, description, path):\n{metadata_json}\n\n"
    
    if message:
        user_input += f"User message: {message}\n"
    if file_paths:
        user_input += "Files uploaded:\n" + "\n".join(str(p) for p in file_paths)

    # Run agent
    reply, files_to_show = run_agent_sync(user_input)

    # Prepare clickable URLs
    files_info = []
    for f in files_to_show:
        meta = next((m for m in metadata if Path(m["path"]) == f), None)
        if meta:
            category = meta.get("category")
            tag = meta.get("tag")
            files_info.append({
                "name": meta["name"],
                "category": category,
                "tag": tag,
                "url": f"/download/{meta['name']}?category={category}&tag={tag}"
            })

    return JSONResponse({
        "chat_reply": reply,
        "files": files_info
    })

@app.get("/files")
def get_files():
    absolute_path = "/Users/vishwajithp/storage/file_metadata.json"
    try:
        with open(absolute_path, "r") as f:
            data = json.load(f)
        return {"files": data}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})
# -----------------------------
# Download endpoint
# -----------------------------
@app.get("/download/{file_name}")
async def download_file(file_name: str, category: str=None, tag: str=None):
    files=fetch_files_for_api(name=file_name, category=category, tag=tag).get("files",[])
    if not files:
        return JSONResponse({"error":"File not found"}, status_code=404)
    
    file_path=Path(files[0]["path"])
    if not file_path.exists():
        return JSONResponse({"error":"File not found on disk"}, status_code=404)

    return FileResponse(path=file_path, filename=files[0]["name"])
# -----------------------------
# Front-end
# -----------------------------
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>File Manager Agent</title>
<style>
  body { font-family: Arial, sans-serif; padding: 20px; }
  .chat { margin-bottom: 10px; }
  .file-links { margin-top: 10px; }
  .uploaded-list { margin-top: 10px; color: green; }
</style>
</head>
<body>

<h2>🤖 File Manager Agent</h2>

<form id="chatForm">
  <div class="chat">
    <label>Message:</label><br>
    <input type="text" id="message" size="50">
  </div>
  <div class="chat">
    <label>Upload Files (optional):</label><br>
    <input type="file" id="file" multiple>
  </div>
  <button type="submit">Send</button>
</form>

<h3>Uploaded Files:</h3>
<div id="uploadedFiles" class="uploaded-list"></div>

<h3>Reply:</h3>
<div id="reply"></div>

<h3>Files:</h3>
<div id="files" class="file-links"></div>

<script>
const form = document.getElementById('chatForm');
const fileInput = document.getElementById('file');
const uploadedFilesDiv = document.getElementById('uploadedFiles');

fileInput.addEventListener('change', () => {
  uploadedFilesDiv.innerHTML = '';
  if (fileInput.files.length > 0) {
    uploadedFilesDiv.innerHTML = `<strong>${fileInput.files.length} file(s) selected:</strong><br>`;
    for (let i = 0; i < fileInput.files.length; i++) {
      uploadedFilesDiv.innerHTML += `• ${fileInput.files[i].name}<br>`;
    }
  } else {
    uploadedFilesDiv.innerHTML = 'No files selected';
  }
});

form.addEventListener('submit', async (e) => {
  e.preventDefault();

  const formData = new FormData();
  formData.append('message', document.getElementById('message').value);

  if (fileInput.files.length > 0) {
    for (let i = 0; i < fileInput.files.length; i++) {
      formData.append('uploaded_files', fileInput.files[i]);
    }
  }

  const response = await fetch('/chat', {
    method: 'POST',
    body: formData
  });

  const data = await response.json();
  document.getElementById('reply').innerText = data.chat_reply;

  const filesDiv = document.getElementById('files');
  filesDiv.innerHTML = '';
  if (data.files && data.files.length > 0) {
    data.files.forEach(f => {
      const link = document.createElement('a');
      link.href = f.url;
      link.innerText = f.name;
      link.target = '_blank';
      filesDiv.appendChild(link);
      filesDiv.appendChild(document.createElement('br'));
    });
  }

  // --- Clear file input, uploaded files list, and optionally text ---
  fileInput.value = "";
  uploadedFilesDiv.innerHTML = 'No files selected';
  document.getElementById('message').value = "";
});
</script>

</body>
</html>
    """

# -----------------------------
# Run server
# -----------------------------
import asyncio
from uvicorn import Config, Server

async def main():
    config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = Server(config)
    await server.serve()

if __name__ == "__main__":
    asyncio.run(main())


INFO:     Started server process [43425]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59297 - "GET /files HTTP/1.1" 200 OK
INFO:     127.0.0.1:59350 - "GET /files HTTP/1.1" 200 OK
INFO:     127.0.0.1:59360 - "GET /files HTTP/1.1" 200 OK
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/Uwxa842m.pdf
⚠️ File not ready: /Users/vishwajithp/Downloads/Uwxa842m.pdf
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/9781474402095_Three_Short_Novels_Introduction.pdf
INFO:     127.0.0.1:59404 - "POST /chat HTTP/1.1" 200 OK
[AGENT RESPONSE] The file has been successfully added with the following metadata:

- **Name:** 9781474402095_Three_Short_Novels_Introduction.pdf
- **Category:** novels
- **Tag:** short_stories
- **Description:** The document is an introduction to a collection of three short novels by John Galt, showcasing his versatility in themes and styles, along with historical context and annotations to enhance understanding.
- **Path:** /Users/vishwajithp/storage/uploads/9781474402095_Three_Short_Novels_Introduction.pdf

If you need any further actions, j

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [43425]


KeyboardInterrupt: 

[NEW FILE DETECTED] /Users/vishwajithp/Downloads/GVhoutx4.pdf
⚠️ File not ready: /Users/vishwajithp/Downloads/GVhoutx4.pdf
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/FALLSEM2025-26_VL_BCSE302L_00100_TH_2025-09-16_Indxing_-Hashing-and-Optimization.pdf
[ERROR running agent] HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11184c050>: Failed to establish a new connection: [Errno 61] Connection refused'))
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/Re1xFqYG.pdf
⚠️ File not ready: /Users/vishwajithp/Downloads/Re1xFqYG.pdf
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/FALLSEM2025-26_VL_BCSE302L_00100_TH_2025-09-16_File-organizatrion.pdf
⚠️ File not ready: /Users/vishwajithp/Downloads/FALLSEM2025-26_VL_BCSE302L_00100_TH_2025-09-16_File-organizatrion.pdf
[NEW FILE DETECTED] /Users/vishwajithp/Downloads/OuSj6o1C.pdf
⚠️ File not ready: /Users/vishwajithp/Downloads/OuSj